In [1]:
# Install
!pip -q install transformers datasets accelerate seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [1]:
import torch; print("cuda?", torch.cuda.is_available())


cuda? True


In [7]:
# MINIMAL Week 40: span-based QA (ar/ko/te) with k=3 backbones

import pandas as pd, numpy as np, torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer

# --- data ---
langs = ["ar","ko","te"]
splits = {'train': 'train.parquet', 'validation': 'validation.parquet'}
df_train = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["train"])
df_val   = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["validation"])
df_train = df_train[df_train.lang.isin(langs)].reset_index(drop=True)
df_val   = df_val[df_val.lang.isin(langs)].reset_index(drop=True)

def to_hf(df):  # keep only needed columns
    keep = ["lang","question","context","answerable","answer_start","answer"]
    return Dataset.from_pandas(df[keep], preserve_index=False)

ds_tr, ds_va = to_hf(df_train), to_hf(df_val)

# --- preprocessing to SQuAD-style start/end ---
def build_preprocess(tokenizer, max_len=384, stride=128):
    cls_id = tokenizer.cls_token_id
    def _prep(ex):
        q = ex["question"]; c = ex["context"]
        ans_text = ex["answer"] if ex["answerable"] else ""
        ans_start = ex["answer_start"] if ex["answerable"] else -1

        enc = tokenizer(
            q, c,
            max_length=max_len, truncation="only_second", stride=stride,
            return_offsets_mapping=True, return_token_type_ids=True
        )
        # default to CLS (unanswerable or lost by truncation)
        start = end = enc["input_ids"].index(cls_id)

        if ex["answerable"]:
            # find token span inside the context segment
            seq_ids = enc.sequence_ids()
            offsets = enc["offset_mapping"]
            # locate the first context token idx
            c_tok_idxs = [i for i,s in enumerate(seq_ids) if s==1]
            if c_tok_idxs:
                c0, c1 = c_tok_idxs[0], c_tok_idxs[-1]
                # character indexes of answer
                a0, a1 = ans_start, ans_start + len(ans_text)
                # move start forward to the token whose span starts after/before a0
                i = c0
                while i<=c1 and (offsets[i][0] <= a0 and offsets[i][1] <= a0):
                    i += 1
                # back up one step if we stepped past
                while i>c0 and offsets[i-1][0] <= a0 < offsets[i-1][1]:
                    i -= 1
                # if inside range, expand to cover full answer
                if c0 <= i <= c1:
                    j = i
                    while j<=c1 and offsets[j][0] < a1:
                        j += 1
                    start, end = i, min(j-1, c1)

        enc["start_positions"] = start
        enc["end_positions"]   = end
        enc.pop("offset_mapping")
        return enc
    return _prep

# --- tiny exact match / F1 on tokens (very minimal) ---
def simple_metrics(eval_preds, tokenizer=None):
    preds_start = np.argmax(eval_preds.predictions[0], -1)
    preds_end   = np.argmax(eval_preds.predictions[1], -1)

    # alte Version: label_ids ist Tupel (start, end)
    # neue Version: label_ids ist Dict
    labels = eval_preds.label_ids
    if isinstance(labels, dict):
        labels_start, labels_end = labels["start_positions"], labels["end_positions"]
    else:
        labels_start, labels_end = labels  # Tupel

    em = np.mean((preds_start == labels_start) & (preds_end == labels_end))

    # token-level overlap F1 (approx)
    f1s=[]
    for ps,pe,ls,le in zip(preds_start,preds_end,labels_start,labels_end):
        P=set(range(ps,pe+1)) if pe>=ps else set()
        G=set(range(ls,le+1)) if le>=ls else set()
        if not P and not G: f1=1.0
        elif not P or not G: f1=0.0
        else:
            prec = len(P&G)/len(P); rec = len(P&G)/len(G)
            f1 = 0 if (prec+rec)==0 else 2*prec*rec/(prec+rec)
        f1s.append(f1)

    return {"em": float(em), "f1_token": float(np.mean(f1s))}


# --- k=3 backbones (change as you like) ---
MODELS = [
  "google-bert/bert-base-multilingual-cased",
  "distilbert/distilbert-base-multilingual-cased",
  "xlm-roberta-base"
]

for mname in MODELS:
    print("\n=== Training:", mname, "===")
    tok = AutoTokenizer.from_pretrained(mname, use_fast=True)
    prep = build_preprocess(tok)

    tr_enc = ds_tr.map(prep, remove_columns=ds_tr.column_names)
    va_enc = ds_va.map(prep, remove_columns=ds_va.column_names)

    model = AutoModelForQuestionAnswering.from_pretrained(mname)

    args = TrainingArguments(
        output_dir=f"wk40_{mname.split('/')[-1]}",
        learning_rate=5e-5, per_device_train_batch_size=8, per_device_eval_batch_size=8,
        num_train_epochs=1, weight_decay=0.01, logging_steps=100, report_to="none",
    )

    def compute_metrics(p): return simple_metrics(p, tok)

    trainer = Trainer(
        model=model, args=args, tokenizer=tok,
        train_dataset=tr_enc, eval_dataset=va_enc,
        compute_metrics=compute_metrics
    )
    trainer.train()
    print(trainer.evaluate())



=== Training: google-bert/bert-base-multilingual-cased ===


Map:   0%|          | 0/6335 [00:00<?, ? examples/s]

Map:   0%|          | 0/1155 [00:00<?, ? examples/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-4181024445.py:122: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
100,2.974200
200,2.239800
300,1.987900
400,1.635000
500,1.724900
600,1.625600
700,1.540400


...

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-4181024445.py:122: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
100,3.280400
200,2.692000
300,2.629000
400,2.412200
500,2.416000
600,2.383500
700,2.256800


Step,Training Loss
100,3.280400
200,2.692000
300,2.629000
400,2.412200
500,2.416000
600,2.383500
700,2.256800


{'eval_loss': 2.3212661743164062, 'eval_em': 0.2909090909090909, 'eval_f1_token': 0.36364621791364693, 'eval_runtime': 18.2604, 'eval_samples_per_second': 63.252, 'eval_steps_per_second': 7.941, 'epoch': 1.0}


In [9]:
# MINIMAL Week 40: span-based QA (ar/ko/te) with k=3 backbones

import pandas as pd, numpy as np, torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer

# --- data ---
langs = ["ar","ko","te"]
splits = {'train': 'train.parquet', 'validation': 'validation.parquet'}
df_train = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["train"])
df_val   = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["validation"])
df_train = df_train[df_train.lang.isin(langs)].reset_index(drop=True)
df_val   = df_val[df_val.lang.isin(langs)].reset_index(drop=True)

def to_hf(df):  # keep only needed columns
    keep = ["lang","question","context","answerable","answer_start","answer"]
    return Dataset.from_pandas(df[keep], preserve_index=False)

ds_tr, ds_va = to_hf(df_train), to_hf(df_val)

# --- preprocessing to SQuAD-style start/end ---
def build_preprocess(tokenizer, max_len=384, stride=128):
    cls_id = tokenizer.cls_token_id
    def _prep(ex):
        q = ex["question"]; c = ex["context"]
        ans_text  = ex["answer"] if ex["answerable"] else ""
        ans_start = ex["answer_start"] if ex["answerable"] else -1

        enc = tokenizer(
            q, c,
            max_length=max_len, truncation="only_second", stride=stride,
            return_offsets_mapping=True, return_token_type_ids=True
        )
        # default to CLS (unanswerable or lost by truncation)
        start = end = enc["input_ids"].index(cls_id)

        if ex["answerable"]:
            seq_ids = enc.sequence_ids()
            offsets = enc["offset_mapping"]
            c_tok_idxs = [i for i,s in enumerate(seq_ids) if s==1]
            if c_tok_idxs:
                c0, c1 = c_tok_idxs[0], c_tok_idxs[-1]
                a0, a1 = ans_start, ans_start + len(ans_text)
                i = c0
                while i<=c1 and (offsets[i][0] <= a0 and offsets[i][1] <= a0):
                    i += 1
                while i>c0 and offsets[i-1][0] <= a0 < offsets[i-1][1]:
                    i -= 1
                if c0 <= i <= c1:
                    j = i
                    while j<=c1 and offsets[j][0] < a1:
                        j += 1
                    start, end = i, min(j-1, c1)

        enc["start_positions"] = start
        enc["end_positions"]   = end
        enc.pop("offset_mapping")
        return enc
    return _prep

# --- sequence-labelling metrics (EM + Token-F1) mit leerem Output für Unanswerable ---
# Annahme: CLS-Token steht an Position 0 (gilt für BERT/XLM-R Standard-Tokenisierung).
def simple_metrics(eval_preds, tokenizer=None):
    ps = np.argmax(eval_preds.predictions[0], -1)
    pe = np.argmax(eval_preds.predictions[1], -1)

    labels = eval_preds.label_ids
    if isinstance(labels, dict):
        ls, le = labels["start_positions"], labels["end_positions"]
    else:
        ls, le = labels  # (start, end)

    def span_to_set(s, e):
        # Unanswerable: leere Menge (Aufgabenforderung)
        if s==0 and e==0:
            return set()
        if e < s:
            return set()
        return set(range(int(s), int(e)+1))

    # EM (exakt gleiche Token-Menge)
    em_list, f1_list = [], []
    for a,b,c,d in zip(ps,pe,ls,le):
        P, G = span_to_set(a,b), span_to_set(c,d)
        em_list.append(P == G)
        if not P and not G:
            f1 = 1.0
        elif not P or not G:
            f1 = 0.0
        else:
            inter = len(P & G)
            prec  = inter/len(P)
            rec   = inter/len(G)
            f1    = 0.0 if (prec+rec)==0 else 2*prec*rec/(prec+rec)
        f1_list.append(f1)

    return {"em": float(np.mean(em_list)), "f1_token": float(np.mean(f1_list))}

# --- k=3 backbones ---
MODELS = [
  "google-bert/bert-base-multilingual-cased",
  "distilbert/distilbert-base-multilingual-cased",
  "xlm-roberta-base"
]

for mname in MODELS:
    print("\n=== Training:", mname, "===")
    tok = AutoTokenizer.from_pretrained(mname, use_fast=True)
    prep = build_preprocess(tok)

    tr_enc = ds_tr.map(prep, remove_columns=ds_tr.column_names)
    va_enc = ds_va.map(prep, remove_columns=ds_va.column_names)

    model = AutoModelForQuestionAnswering.from_pretrained(mname)

    args = TrainingArguments(
        output_dir=f"wk40_{mname.split('/')[-1]}",
        learning_rate=5e-5, per_device_train_batch_size=8, per_device_eval_batch_size=8,
        num_train_epochs=1, weight_decay=0.01, logging_steps=100, do_eval=True, report_to=[]
        # ggf. alte HF-Version: 'report_to'/'evaluation_strategy' entfernen
    )

    trainer = Trainer(
        model=model, args=args, tokenizer=tok,
        train_dataset=tr_enc, eval_dataset=va_enc,
        compute_metrics=simple_metrics
    )

    trainer.train()
    print("VAL (all):", trainer.evaluate())

    # --- Pflicht: pro Sprache evaluieren und vergleichen ---
    for L in langs:
        va_L = ds_va.filter(lambda ex, L=L: ex["lang"]==L)
        va_L_enc = va_L.map(prep, remove_columns=va_L.column_names)
        res_L = trainer.evaluate(eval_dataset=va_L_enc)
        print(f"VAL [{L}]:", res_L)



=== Training: google-bert/bert-base-multilingual-cased ===


Map:   0%|          | 0/6335 [00:00<?, ? examples/s]

Map:   0%|          | 0/1155 [00:00<?, ? examples/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1869919114.py:123: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
100,2.828500
200,2.022800
300,1.934000
400,1.646600
500,1.690700
600,1.604000
700,1.530300


...

Map:   0%|          | 0/356 [00:00<?, ? examples/s]

VAL [ko]: {'eval_loss': 1.9591587781906128, 'eval_em': 0.3792134831460674, 'eval_f1_token': 0.4522927361598218, 'eval_runtime': 5.8349, 'eval_samples_per_second': 61.013, 'eval_steps_per_second': 7.712, 'epoch': 1.0}


Filter:   0%|          | 0/1155 [00:00<?, ? examples/s]

Map:   0%|          | 0/384 [00:00<?, ? examples/s]

VAL [te]: {'eval_loss': 2.527325391769409, 'eval_em': 0.3020833333333333, 'eval_f1_token': 0.3778042272565769, 'eval_runtime': 6.149, 'eval_samples_per_second': 62.449, 'eval_steps_per_second': 7.806, 'epoch': 1.0}
